# Data Processing

## import packages

In [1]:
import numpy as np
import pandas as pd
from arbitragerepair import constraints, repair

In [2]:
raw_data = pd.read_csv('fqjo3s8eacwzxkcw.csv')
tickers = raw_data['ticker'].unique()

,date,exdate,cp_flag,strike_price,volume,impl_volatility,option_price
0,2018-01-02,2018-01-05,C,1000.0,0,0.962943,189.975
1,2018-01-02,2018-01-05,C,1002.5,0,0.951017,187.475
2,2018-01-02,2018-01-05,C,1005.0,0,0.939104,184.975
3,2018-01-02,2018-01-05,C,1007.5,0,0.927205,182.475
4,2018-01-02,2018-01-05,C,1010.0,0,0.915319,179.975


## Revise stock prices

### Upload stock data


In [32]:
import datetime as dt
import yfinance as yf

## download split history
splits_data = pd.DataFrame()
for ticker in tickers:
    stock = yf.Ticker(ticker)
    splits = stock.splits
    splits.name = ticker  
    splits_data = splits_data.join(splits, how='outer')  
splits_data = splits_data[splits_data.index >= '2018-01-01']


for ticker in tickers:
    
    stock = yf.Ticker(ticker)
    start_date = dt.datetime(2018, 1, 1)
    tickers_list = [ticker]
    df_stock = yf.download(tickers_list, start=start_date, progress=True)
    
    df_stock.reset_index(inplace=True)
    df_stock['date'] = pd.to_datetime(df_stock['Date'], format="%d/%m/%y")
    df_stock = df_stock[['date','Adj Close']].copy()
    df_stock.rename(columns = {'Adj Close' : 'adjusted_price'}, inplace = True)

    split_stock = splits_data[ticker].dropna()
    if len(split_stock) != 0:
        for date in split_stock.index:
            mask = df_stock['date'] < date.tz_localize(None)
            df_stock.loc[mask, 'adjusted_price'] *= split_stock.loc[date]

    fn = './data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower())
    df_stock.to_csv(fn, sep=',', encoding='utf-8')
    print('saved to %s'%(fn))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


saved to ./data/adjusted_Stock_Daily/amzn_stock_daily_adjusted.csv
saved to ./data/adjusted_Stock_Daily/jpm_stock_daily_adjusted.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


saved to ./data/adjusted_Stock_Daily/jnj_stock_daily_adjusted.csv
saved to ./data/adjusted_Stock_Daily/msft_stock_daily_adjusted.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

saved to ./data/adjusted_Stock_Daily/pg_stock_daily_adjusted.csv



[*********************100%%**********************]  1 of 1 completed


saved to ./data/adjusted_Stock_Daily/wmt_stock_daily_adjusted.csv
saved to ./data/adjusted_Stock_Daily/googl_stock_daily_adjusted.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

saved to ./data/adjusted_Stock_Daily/v_stock_daily_adjusted.csv
saved to ./data/adjusted_Stock_Daily/tsla_stock_daily_adjusted.csv


### Process different pairs of data, with 3 months difference for T2 and T1

In [53]:
# Arbitrage repair function
# pip install pandas_market_calendars
import pandas_market_calendars as mcal
def arbitrageRepair(columnT, columnK, columnC, columnF):
    # normalise strikes and call prices
    normaliser = constraints.Normalise()
    T = np.array(columnT)
    K = np.array(columnK)
    C = np.array(columnC)
    F = np.array(columnF)
    normaliser.fit(T, K, C, F)
    T1, K1, C1 = normaliser.transform(T, K, C)
    
    # construct arbitrage constraints and detect violation
    mat_A, vec_b, _, _ = constraints.detect(T1, K1, C1, verbose=False)
    
    # repair arbitrage - l1-norm objective
    epsilon = repair.l1(mat_A, vec_b, C1)
    
    # de-normalise
    K0, C0 = normaliser.inverse_transform(K1, C1 + epsilon)
    
    return K0, C0


def dataProcess2Di(df,t1,t2,ticker,stock_daily):

    # call options data
    df_t1_C = df.loc[(df.exdate==t1)&(df.cp_flag=='C'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]
    df_t2_C = df.loc[(df.exdate==t2)&(df.cp_flag=='C'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]

    # put options data
    df_t1_P = df.loc[(df.exdate==t1)&(df.cp_flag=='P'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]
    df_t2_P = df.loc[(df.exdate==t2)&(df.cp_flag=='P'), ['date','exdate','strike_price','volume','option_price','impl_volatility']]

    t0List = list(df_t1_C.date.unique())

    result = pd.DataFrame()

    for t0 in t0List:
        ## T1:
        # t1: 20 largest trading volume
        df_t0t1 = df_t1_C[df_t1_C.date==t0].sort_values('volume',ascending=False).iloc[0:20].drop(columns='volume').reset_index(drop=True)

        # merge with put options data and stock price data
        df_t0t1 = df_t0t1.merge(df_t1_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t1 = pd.merge(df_t0t1, stock_daily, on = 'date')

        # calculate the forward price:
        df_t0t1['forward_price'] = df_t0t1['adjusted_price']/((df_t0t1['adjusted_price']-(df_t0t1['option_price_C']-df_t0t1['option_price_P']))/df_t0t1['strike_price'])
        #df_t0t1['forward_price'] = df_t0t1['adjusted_price']

        # calculate the expiry:
        # Amazon (AMZN) is listed on the NASDAQ exchange
        amzn_t = mcal.get_calendar('NASDAQ')
        trading_days_1 = amzn_t.valid_days(start_date=t0, end_date=t1)
        df_t0t1['expiry'] = len(trading_days_1) / 252
        #df_t0t1['expiry'] = (df_t0t1['exdate']-df_t0t1['date']).apply(lambda x: x.days/365)

        # arbitrage repair:
        if len(df_t0t1)==0:
            continue
        K0_t1, C0_t1 = arbitrageRepair(df_t0t1['expiry'], df_t0t1['strike_price'], df_t0t1['option_price_C'], df_t0t1['forward_price'])

        # temporary result at t0:
        result_t1 = df_t0t1[['date','exdate']].copy()
        result_t1['strike_price'] = K0_t1
        result_t1['call_option_price'] = C0_t1


        ## T2:
        # t2: 20 largest trading volume
        df_t0t2 = df_t2_C[df_t2_C.date==t0].sort_values('volume',ascending=False).iloc[0:20].drop(columns='volume').reset_index(drop=True)

        # merge with put options data and stock price data
        df_t0t2 = pd.merge(df_t0t2, df_t2_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t2 = pd.merge(df_t0t2, stock_daily, on = 'date')

        # calculate the forward price:
        df_t0t2['forward_price'] = df_t0t2['adjusted_price']/((df_t0t2['adjusted_price']-(df_t0t2['option_price_C']-df_t0t2['option_price_P']))/df_t0t2['strike_price'])
        # df_t0t2['forward_price'] = df_t0t2['adjusted_price']

        # calculate the expiry:
        trading_days_2 = amzn_t.valid_days(start_date=t0, end_date=t2)
        df_t0t2['expiry'] = len(trading_days_2) / 252
        # df_t0t2['expiry'] = (df_t0t2['exdate']-df_t0t2['date']).apply(lambda x: x.days/365)

        # arbitrage repair:
        if len(df_t0t2)==0:
            continue
        K0_t2, C0_t2 = arbitrageRepair(df_t0t2['expiry'], df_t0t2['strike_price'], df_t0t2['option_price_C'], df_t0t2['forward_price'])

        # temporary result at t0:
        result_t2 = df_t0t2[['date','exdate']].copy()
        result_t2['strike_price'] = K0_t2
        result_t2['call_option_price'] = C0_t2


        ## Concat horizontally  
        result_tmp = pd.concat([result_t1, result_t2.drop(columns='date')], axis=1)

        ## Concat vertically
        result = pd.concat([result,result_tmp], axis=0)

    result.columns = ['date', 'T1', 'K1', 'C1',  'T2', 'K2', 'C2']
    result = result.reset_index(drop=True)
    result = pd.merge(result, stock_daily, on = 'date')
#     import pdb
#     pdb.set_trace()
    result.columns = ['t0', 'T1', 'K1', 'C1',  'T2', 'K2', 'C2',  'Adj_S0']
    file_name = 'data/paired_Option/{}_{}_{}.csv'.format(ticker,pd.to_datetime(t1).strftime('%Y%m%d'),pd.to_datetime(t2).strftime('%Y%m%d'))
    result.to_csv(file_name, header=True)

In [54]:
import warnings
warnings.filterwarnings('ignore')
for ticker in tickers:
    stock_Daily = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower()),index_col=0)
    stock_Daily['date'] = pd.to_datetime(stock_Daily['date'])
    df = raw_data.loc[raw_data.ticker==ticker, 
                      ['date','exdate','cp_flag','strike_price','best_bid','best_offer','volume','impl_volatility']].copy()
    df['date'] = pd.to_datetime(df['date'])
    df['exdate'] = pd.to_datetime(df['exdate'])
    df['strike_price'] = df['strike_price']/1000 # scaling strike
    df['option_price'] = df[['best_bid','best_offer']].mean(axis=1)
    df = df.drop(columns = ['best_bid','best_offer'])
    df.head()

    ## Select T1 and T2 which has closest difference of 3 months, arbitrage repair and save to path 'data/'
    T1_List = df['exdate'].unique()
    T1_T2_List = pd.DataFrame()
    for t1 in T1_List:
        t2 = T1_List[T1_List.year*12+T1_List.month == t1.year*12+t1.month+1]
        if len(t2) > 1:
            t2 = t2[t2.day == min(np.abs(t2.day-t1.day))]
        if len(t2) > 0:
            T1_T2_List = pd.concat([T1_T2_List,pd.DataFrame([t1,t2[0]]).T])
    for t2 in T1_T2_List[1]:
        if len(T1_T2_List[T1_T2_List[1]==t2]) >1:
            df_t2 = T1_T2_List[T1_T2_List[1]==t2]
            df_t2['diff'] = np.abs(df_t2[0].dt.day - t2.day)
            df_t2 = df_t2.sort_values(by='diff').iloc[:1,:2]
            T1_T2_List = pd.concat([T1_T2_List[T1_T2_List[1]!=t2],df_t2])

    for i in range(len(T1_T2_List)):
        t1 = T1_T2_List.iloc[i,0]
        t2 = T1_T2_List.iloc[i,1]
        dataProcess2Di(df,t1,t2,ticker=ticker,stock_daily=stock_Daily)